### Problem Statement
Estimate individual medical insurance expenses based on factors such as age, BMI, smoking status, and other health-related variables.

### Data Description
**Dataset Filename:** `data_insurance.csv`

#### Attributes Information:
1. **age:** Age of the primary policyholder.
2. **sex:** Gender of the insurance policyholder (female or male).
3. **bmi:** Body Mass Index, a measure that uses the ratio of weight to height (kg/m²) to categorize body weight, ideally ranging from 18.5 to 24.9.
4. **children:** Number of dependents covered by the health insurance.
5. **smoker:** Indicates if the person smokes.
6. **region:** The geographical area in the U.S. where the policyholder resides (northeast, southeast, southwest, northwest).

**Target:**
- **charges:** Medical expenses billed to the insurance for the individual.
vidual.rice of unit area

In [200]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import math
import random

### Reading the data
Here we read the data from the file and store it in a pandas dataframe.

In [203]:
filepath = 'data_insurance.csv'

df = pd.read_csv(filepath)

### Explore the data
We explore the first few rows of the dataset and describe it

In [206]:
print("First few rows of the dataset:")
print(df.head())

print("\nSummary statistics:")
print(df.describe())

print("\nData types of each column:")
print(df.dtypes)

First few rows of the dataset:
   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520

Summary statistics:
               age          bmi     children       charges
count  1338.000000  1338.000000  1338.000000   1338.000000
mean     39.207025    30.663397     1.094918  13270.422265
std      14.049960     6.098187     1.205493  12110.011237
min      18.000000    15.960000     0.000000   1121.873900
25%      27.000000    26.296250     0.000000   4740.287150
50%      39.000000    30.400000     1.000000   9382.033000
75%      51.000000    34.693750     2.000000  16639.912515
max      64.000000    53.130000     5.000000  63770.428010

Data types of each column:
age  

### Handling missing data
1. Option 1: Fill missing values with a specific value (e.g., mean)
2. Option 2: Drop rows/columns with missing values

In [209]:
missing_values = df.isnull().sum()
print("\nMissing values in each column:")
print(missing_values)

if missing_values.sum() > 0:
    df.fillna(df.mean(), inplace=True)
    print("\nMissing values were found and have been filled with the mean of each column.")

    #df.dropna(inplace=True)
    #print("\nMissing values were found and have been dropped.")
    
    print("\nMissing values after handling:")
    print(df.isnull().sum())
else:
    print("\nNo missing values found in the dataset.")


Missing values in each column:
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

No missing values found in the dataset.


### Converting Categorical Variables
We convert non-numeric features like 'sex', 'smoker' and 'region' into numerical values using encoding techniques.

Since 'sex' and 'smoker' have binary values, we use **label_encoding** on them. \
'region' has multiple values, hence we use **one-hot encoding**.

In [212]:
label_encoding_map = {
    'sex': {'female': 0, 'male': 1},
    'smoker': {'no': 0, 'yes': 1},
    'region': {'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3}
}

df['sex'] = df['sex'].map(label_encoding_map['sex'])
df['smoker'] = df['smoker'].map(label_encoding_map['smoker'])
df['region'] = df['region'].map(label_encoding_map['region'])

print("\nDataset after encoding categorical variables:")
print(df.head())
# print(df.dtypes)


Dataset after encoding categorical variables:
   age  sex     bmi  children  smoker  region      charges
0   19    0  27.900         0       1       0  16884.92400
1   18    1  33.770         1       0       1   1725.55230
2   28    1  33.000         3       0       1   4449.46200
3   33    1  22.705         0       0       2  21984.47061
4   32    1  28.880         0       0       2   3866.85520


### Defining Features and Targets
We create 2 numpy arrays $X$ and $y$ from the dataframe:
+ $X$: Input data of the shape (number of samples, number of input features)
+ $y$: Target variable of the shape (number of samples,)

In [215]:
X = df.drop(columns=['charges']).values
y = df['charges'].values

print("Shape of X: ", X.shape, ", Shape of y: ", y.shape)

Shape of X:  (1338, 6) , Shape of y:  (1338,)


### Splitting the Dataset
We need to pre process the data. We use min-max scaler to scale the input data $X$.
After that, we split the data (**X** and **y**) into a training dataset (**X_train** and **y_train**) and test dataset (**X_test** and **y_test**).

In [218]:
def train_test_split(X, y, test_size=0.25, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    split_index = int(X.shape[0] * (1 - test_size))

    train_indices = indices[:split_index]
    test_indices = indices[split_index:]

    X_train = X[train_indices]
    X_test = X[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]

    return X_train, X_test, y_train, y_test

def min_max_scaler(X, feature_range=(0, 1)):
    X_min = np.min(X, axis=0)
    X_max = np.max(X, axis=0)

    X_scaled = (X-X_min)/(X_max-X_min)

    return X_scaled


X = min_max_scaler(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print("Shape of X_train: ",X_train.shape, ", Shape of y_train: ",y_train.shape)
print("Shape of X_test: ",X_test.shape, ", Shape of y_test: ",y_test.shape)

Shape of X_train:  (1003, 6) , Shape of y_train:  (1003,)
Shape of X_test:  (335, 6) , Shape of y_test:  (335,)


### Computing the loss function
In linear regression, the model parameters are:

+ $w$: Parameters of the linear regression model (excluding the bias) of the shape (1, number of features)

+ $b$: Bias parameter (scalar) of the linear regression model

Both $w$ and $b$ are numpy arrays.

Given the model parameters $w$ and $b$, the prediction for an input sample $X^i$ is:
$$h_{w,b}(X^i) = w \cdot X^i + b$$
where $X^i$ is the $i^{th}$ training sample with shape (number of features,1)

We implement and compute Mean Squarred Error loss fucntion:
$$ L_{w,b}(X) = (1/(2m))\sum_{i=1}^{m}(y^i - h_{w,b}(X^i))^2 $$
where $y^i$ is the true target value for the $i^{th}$ sample and $h_{w,b}(X^i)$ is the predicted value for the $i^{th}$ sample using the parameters $w$ and $b$.

$w$ is the list of parameters excluding the bias and $b$ is the bias term.

In [221]:
def loss_function(X, y, w, b):
    m = X.shape[0]

    loss = 0
    for i in range(m):
        loss += (y[i] - (np.dot(X[i], w) + b)) ** 2        

    loss = (1 / (2 * m)) * loss
    return loss

### Computing the Gradient of the Loss

In this following function ```compute_gradient```, we compute the gradients $\frac{\partial L}{\partial w}$ and $\frac{\partial L}{\partial b}$ of the loss $L$ w.r.t. $w$ and $b$. More specifically, we iterate over every training example and compute the gradients of the loss for that training example. Finally, we aggregate the gradient values for all the training examples and take the average. The gradients can be computed as:
$$\frac{\partial L}{\partial w} = \frac{1}{m} \sum_{i=1}^m (h_{w,b}(X^i)-y^i)X^i$$

$$\frac{\partial L}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} (h_{w,b}(X^i)-y^i)$$


In [224]:
def compute_gradient(X, y, w, b):
    m = X.shape[0]

    dL_dw = None
    dL_db = None

    n = w.shape[0]
    dL_dw = np.zeros((n, ))
    dL_db = 0.0

    for i in range(m):
        diff = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            dL_dw[j] += diff * X[i, j].item()
        dL_db += diff

    dL_dw /= m
    dL_db /= m
    return dL_dw, dL_db

### Training the Model using Batch Gradient Descent
We finally implement the batch gradient descent algorithm to train and learn the parameters of the linear regression model. We use the **loss_function** and **compute_gradient** functions implemented above.

In this batch_gradient_descent function, we compute the gradient for the training samples and update the parameters $w$ and $b$ in every iteration:

+ $w \leftarrow w - \alpha \frac{\partial L}{\partial w}$

+ $b \leftarrow b - \alpha \frac{\partial L}{\partial b}$

Additionally, we compute the loss function values in every iteration and store it in the list variable **loss_hist** and print the loss value after every 100 iterations during the training process.

In [229]:
def batch_gradient_descent(X, y, w_initial, b_initial, alpha, num_iters):
    m = X.shape[0]

    loss_hist = []

    w = copy.deepcopy(w_initial)
    b = b_initial

    for i in range(num_iters):
        dL_dw, dL_db = compute_gradient(X, y, w, b)

        w -= alpha * dL_dw
        b -= alpha * dL_db

        loss = loss_function(X, y, w, b)
        loss_hist.append(loss)

        if (i % 100 == 0):
            print(f"Iteration: {i} Loss: {loss}")

    return w, b, loss_hist

### Initialising the parameters
Now we intialize the model parameters ($w$ and $b$) and learning rate (**alpha**). The learning rate alpha is randomly initialized between 0.0001 and 0.001. For the learning rate, we set a seed and make use of the random function.

In [232]:
random_seed = 88
np.random.seed(random_seed)

def initialize_parameters():
    initial_w = None
    initial_b = None
    alpha = None

    initial_w = np.random.random((X_train.shape[1], ))
    initial_b = np.random.random()
    alpha = random.uniform(0.0001, 0.001)

    return initial_w,initial_b,alpha

### Training the model
The model is trained using batch gradient descent algorithm for **num_iters=10000** iterations. You can change the number of iterations to check any improvements in the performance.

In [235]:
initial_w, initial_b, alpha = initialize_parameters()

num_iters = 10000

w,b,loss_hist = batch_gradient_descent(X_train ,y_train, initial_w, initial_b, alpha, num_iters)
print("Updated w: ",w)
print("Updated b: ",b)

Iteration: 0 Loss: 163477255.74215737
Iteration: 100 Loss: 147543637.56146348
Iteration: 200 Loss: 134004696.38238263
Iteration: 300 Loss: 122489278.01012178
Iteration: 400 Loss: 112683906.1876517
Iteration: 500 Loss: 104323817.31645216
Iteration: 600 Loss: 97185388.73983751
Iteration: 700 Loss: 91079743.97319362
Iteration: 800 Loss: 85847351.93624076
Iteration: 900 Loss: 81353465.6794425
Iteration: 1000 Loss: 77484270.11341716
Iteration: 1100 Loss: 74143628.53378394
Iteration: 1200 Loss: 71250334.86457336
Iteration: 1300 Loss: 68735793.011208
Iteration: 1400 Loss: 66542056.933080256
Iteration: 1500 Loss: 64620175.365407884
Iteration: 1600 Loss: 62928793.83564601
Iteration: 1700 Loss: 61432973.98055398
Iteration: 1800 Loss: 60103196.386684135
Iteration: 1900 Loss: 58914518.427394666
Iteration: 2000 Loss: 57845863.00371844
Iteration: 2100 Loss: 56879417.841392405
Iteration: 2200 Loss: 56000128.159197085
Iteration: 2300 Loss: 55195268.19497706
Iteration: 2400 Loss: 54454079.33171225
Iter

### Evaluating the Model
We assess the model's accuracy using metrics like Mean Squared Error (MSE) and R-squared (R²).

In [238]:
train_error = loss_function(X_train, y_train, w, b)
test_error = loss_function(X_test, y_test, w, b)
print("Train Error: ",train_error, ", Test Error: ",test_error)

def calculate_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def calculate_r2(y_true, y_pred):
    ss_total = np.sum((y_true - np.mean(y_true)) ** 2)
    ss_residual = np.sum((y_true - y_pred) ** 2)
    return 1 - (ss_residual / ss_total)

def predict(X, w, b):
    return np.dot(X, w) + b

y_train_pred = predict(X_train, w, b)
y_test_pred = predict(X_test, w, b)

train_mse = calculate_mse(y_train, y_train_pred)
test_mse = calculate_mse(y_test, y_test_pred)

train_r2 = calculate_r2(y_train, y_train_pred)
test_r2 = calculate_r2(y_test, y_test_pred)

print("Train MSE: ", train_mse, ", Test MSE: ", test_mse)
print("Train R²: ", train_r2, ", Test R²: ", test_r2)

Train Error:  32334284.328970097 , Test Error:  32638524.933168355
Train MSE:  64668568.65794012 , Test MSE:  65277049.866336696
Train R²:  0.5616217966020205 , Test R²:  0.5447525730204511


### Analysing Model Coefficients
We examine the model coefficients to understand the impact of each feature on the predicted
charges

In [241]:
def display_coefficients(features, coefficients, intercept):
    coef_df = pd.DataFrame({
        'Feature': features,
        'Coefficient': coefficients
    })
    coef_df = coef_df.sort_values(by='Coefficient', ascending=False)
    coef_df = coef_df.reset_index(drop=True)
    
    print("\nModel Coefficients:")
    print(coef_df)
    print("\nIntercept (Bias):", intercept)

feature_names = df.drop(columns=['charges']).columns.tolist()

display_coefficients(feature_names, w, b)



Model Coefficients:
    Feature   Coefficient
0    smoker  12243.924090
1       age   5613.446994
2       bmi   3352.220950
3    region   1625.075432
4       sex   1585.792907
5  children   1473.017740

Intercept (Bias): 5294.914406144607
